In [1]:
import sys
import io
import os
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers
from sklearn.metrics import classification_report

# Set the console output to UTF-8
# Paths to Hilbert spectrum image folders
train_data_path = "D:/datasets/Hilbert_spectrum"
dysarthric_folder = os.path.join(train_data_path, "dysarthric")
non_dysarthric_folder = os.path.join(train_data_path, "non_dysarthric")

# Create directories for training and validation sets
train_dir = os.path.join(train_data_path, 'train')
val_dir = os.path.join(train_data_path, 'val')

# Function to split data and create train/val folders if they don't exist
def split_data(source_folder, train_folder, val_folder, split_ratio=0.8):
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    
    # Get list of all files in the source folder
    files = [f for f in os.listdir(source_folder) if f.endswith('.png')]
    train_files, val_files = train_test_split(files, train_size=split_ratio, random_state=42)

    # Copy files to train folder
    for file in train_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(train_folder, file))

    # Copy files to val folder
    for file in val_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(val_folder, file))

# Split data for both classes
split_data(dysarthric_folder, os.path.join(train_dir, 'dysarthric'), os.path.join(val_dir, 'dysarthric'))
split_data(non_dysarthric_folder, os.path.join(train_dir, 'non_dysarthric'), os.path.join(val_dir, 'non_dysarthric'))

# Define the image data generator with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create generators for train and validation datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # EfficientNetB0 default input size
    class_mode='binary',
    batch_size=32
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    class_mode='binary',
    batch_size=32
)

# Load the EfficientNetB0 model with pre-trained ImageNet weights, excluding the top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Build the model
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with progress tracking
epochs = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    verbose=1
)

# Unfreeze the base model for fine-tuning
base_model.trainable = True
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training (fine-tuning)
fine_tune_epochs = 10
total_epochs = epochs + fine_tune_epochs
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    verbose=1
)

# Evaluate the model on the validation set
val_generator.reset()
predictions = model.predict(val_generator)
predicted_classes = np.round(predictions).astype(int)

# Print classification report
print("Classification Report:")
print(classification_report(val_generator.classes, predicted_classes))

# Save the model
model.save("dysarthric_classifier_efficientnet_model.h5")

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'] + history_fine.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'] + history_fine.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'] + history_fine.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'] + history_fine.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()


Found 13416 images belonging to 2 classes.
Found 4235 images belonging to 2 classes.
16705208/16705208 [==============================] - 43s 3us/step
Epoch 1/10
 30/420 [=>............................] - ETA: 22:28 - loss: 0.6985 - accuracy: 0.5146

KeyboardInterrupt: 